In [2]:
import re

### Création de la liste de mots-clés haineux

In [3]:
with open("kw_hate.txt", "r") as f:
    kw = f.read().splitlines()

### Analyse des clusters
- On itère sur chaque méthode de nettoyage et de clustering
- Pour chaque couple de méthodes on analyse le fichier `clusters_info` correspondant
- On affiche le numéro de cluster, le nombre de messages et le pourcentage de messages haineux pour chaque cluster qui :
  - contient au moins 10% de messages contenant un mot-clé haineux
  - ou bien a un mot-clé haineux dans les mots les plus proches de son centre

In [4]:
for clean_method in ["MinStp", "Std", "Lem"]:
    for clust_method in ["km_25", "km_50", "km_100", "som_100", "som_225", "som_400"]:
        dir = "clusters/1M_{}_clusters/{}/".format(clean_method, clust_method)
        filename = clust_method + "_clusters_info.txt"
        print("{} + {}".format(clean_method, clust_method))
        with open(dir + filename, "r") as f:
            lines = f.read().split("\n\n")
        for i, line in enumerate(lines):
            res = re.findall("(\d*\.\d*)%", line)
            n_msg_res = re.findall("(\d*) messages", line)
            if (len(n_msg_res) > 0):
                n_msg = int(n_msg_res[0])
            c = re.findall("centre : .*", line)
            if (len(c) > 0):
                center = c[0][9:]
            if (len(res) > 0):
                num = float(res[0])
            if (num > 3 or any(item in center for item in kw)):
                print("Cluster {} ({} messages) : {}% de messages contiennent un mot haineux".format(i, n_msg, num))
        print()

MinStp + km_25

MinStp + km_50
Cluster 27 (1712 messages) : 0.41% de messages contiennent un mot haineux

MinStp + km_100
Cluster 22 (1515 messages) : 0.33% de messages contiennent un mot haineux
Cluster 23 (1616 messages) : 0.31% de messages contiennent un mot haineux
Cluster 30 (33398 messages) : 1.48% de messages contiennent un mot haineux
Cluster 36 (7784 messages) : 7.17% de messages contiennent un mot haineux
Cluster 59 (1122 messages) : 4.01% de messages contiennent un mot haineux
Cluster 70 (1532 messages) : 1.96% de messages contiennent un mot haineux
Cluster 83 (1670 messages) : 1.02% de messages contiennent un mot haineux
Cluster 94 (3243 messages) : 5.49% de messages contiennent un mot haineux

MinStp + som_100
Cluster 29 (5647 messages) : 4.55% de messages contiennent un mot haineux
Cluster 74 (2186 messages) : 4.3% de messages contiennent un mot haineux
Cluster 76 (2468 messages) : 0.61% de messages contiennent un mot haineux
Cluster 77 (2463 messages) : 1.02% de messages